\title{Génération des données}
\author{Orlane Zang - orlane.zang@facsciences-uy1.cm}
\maketitle

# Constructions de la fonction générant générant la dynamique en fonction du modèle de Lindblad utilisé


In [ ]:
#Différentes importations
from itertools import permutations, product # Itertools est un module qui  implémente un certain nombre 
                                            # de blocs de construction d’itérateurs 
from quantum_heom import bath # Accès aux propriétés du bain
from quantum_heom import utilities as util #Différentes fonctions sont déjà définies dans utilities. Le fichier utilisties
                    #se trouve également dans le Github
import numpy as np #Manipulation des matrices, tableaux multidimensionnels, fonctions mathématiques.

#Liste des 3 modèles de Lindblad
LINDBLAD_MODELS = ['local dephasing lindblad',
                   'global thermalising lindblad',
                   'local thermalising lindblad'] # Ici, je crée une liste de chaine de caractères donc chaque str est le 
                                                # nom d'un des modèles de Lindblad

#Définition des différents opérateurs de Lindblad selon le modèle 

#Modèle du déphasage local

def loc_deph_lindblad_op(dims: int, site_j: int) -> np.ndarray:  #Dans la forme générale de l'équation de Lindblad, ce qui fait
    #la différence entre les 3 modèles se trouve au niveau de la construction des opérateurs de Lindblad. Dans le modèle du dé
    #phasage local, les opérateurs dépendent d'indidices alpha qui représentent des sites i. Alors ici, on définit une fonction
    #qui prend en argument le nombre de sites qui représentent la dimension de l'op stat et donc de l'opérateur de Lindblad qui
    #nous interesse. Le second argument est donc la donnée sur l'indice alpha. La fonction définit, on la convertit en une ma-
    #trice carrée.
    assert site_j >= 0, #Il est important de préciser que le numéro du site qui nous interesse ne peut être négatif
    assert site_j <= dims #et ne peut être supérieur au nombre total de sites
    
    l_op = np.zeros((dims, dims), dtype=complex) #J'initialise mon opérateur de Lindblad avec une matrice nulle de "dims" lignes 
    #(nombre de sites) et "dims" colonnes, et prenant comme éléments des nombres complexes
    l_op[site_j][site_j] = 1.+0.j              #### ici, je précise que le (site_j)ième élément de la
                                                #(site_j)ième colonne doit être égal à 1 
    return l_op

#Modèle dthermalisation globale

def glob_therm_lindblad_op(dims: int, state_a: int, state_b: int):
#Dans ce modèle, l'opérateur de Lindblad est donné par A = \\ket{b} \\bra{a}, avec ket{a}, ket{b} des états propres de H_S tels
            #qu'il y a un transfert de population de ket{a} à ket{b}.
#Ici, dims=N=nombre de sites et donc dimension de l'opérateur.   

assert dims > 1, #Le nombre de sites doit être supérieur à 1
    assert state_a >= 0,#Le nombre représentant l'état ket{a} doit être un nombre entier positif
    assert state_b >= 0, #pareil pour ket{b}
    assert state_a <= dims - 1, #Ces nombres ne peuvent pos être plus grand que le nombre total d'états propres
    assert state_b <= dims - 1, 
    assert state_a != state_b, #Les opérateurs du modèle de thermalisation globale ne se construisent que pour des paires 
    # d'états différents.  != signifie "différent de"

    ket_b = np.zeros(dims, dtype=complex) #A ket{b} j'affecte une matrice nulle de dimension le nombre de sites et prenant des 
                            #nombres complexes comme éléments
    bra_a = np.zeros(dims, dtype=complex) #Pareil pour ket{a}
    
    ket_b[state_b] = 1.                 #Le (state_b - 1)ième élément de ket_b doit être égal à 1
    bra_a[state_a] = 1.                 #Pareil
    
    l_op = np.outer(ket_b, bra_a)    #Expression de l'opérateur de Lindblad dans le modèle de thermali. globale
                                    #numpy.outer est le produit de deux vecteurs
    return l_op

#Modèle de thermalisation locale (MTL)

def loc_therm_lindblad_op(eigv: np.ndarray, eigs: np.ndarray, unique: float,
                          site_m: int) -> np.ndarray:
    #Dans le MTL, l'opértateur est donné par 
    #A = \\sum_{\\omega_{ij}=\\omega}
    #      c_m^*(j) c_m(i) \\ket{j} \\bra{i}. Ce modèle tient compte des étts propres de HS mais également des valeurs propres
    #de ce dernier (\\omega_i et \\omega_j).
    
    dims = len(eigv) #Vu que le nombre de valeurs propres est forcément égal à la dimension de H_S
    l_op = np.zeros((dims, dims), dtype=complex) #Initialisation à une matrice nulle dims \\times dims.
    
    for idx_i, idx_j in product(range(dims), repeat=2):
        omega_i, omega_j = eigv[idx_i], eigv[idx_j]
        state_i, state_j = eigs[:, idx_i], eigs[:, idx_j]
        if omega_i - omega_j == unique:
            l_op += (state_j[site_m].conjugate() * state_i[site_m]
                     * np.outer(state_j, state_i))
    return l_op
    
    #Superoperateur
    
    def lindblad_superop_sum_element(l_op: np.ndarray) -> np.ndarray:

        #Le superopérateur est d'expression A^* \\otimes A - 0.5
        #((A^{\\dagger} A)^* \\otimes I + I \\otimes A^{\\dagger} A)
        
   

    assert l_op.shape[0] == l_op.shape[1],  #Vu que plus haut on a
       #défini les opérateurs comme étant des matrices N fois N, ici on précise que npotre superop 
        #doit avoir la première dimension égale à la deuxième

    l_op_dag = l_op.T.conjugate() #T.conjugate=dag
    iden = np.eye(l_op.shape[0]) #L'identité doit avoir la première dimension de l'opérateur de Lindblad
    
    return (np.kron(l_op.conjugate(), l_op) #Ici, on donne l'expression du superopéatrteur, 
            - 0.5 * (np.kron(np.matmul(l_op_dag, l_op).conjugate(), iden)#Kron() fait le produit tensoriel de ceux entre parenth.
                     + np.kron(iden, np.matmul(l_op_dag, l_op)))) #matmul signifie multiplication mathématique

#Ici, il est question de définir la fonction qui gouvernera la dynamqiue de notre système, selon le modèle utilisé.
def lindbladian_superop(dims: int, dynamics_model: str,
                        hamiltonian: np.ndarray = None, deph_rate: float = None,
                        cutoff_freq: float = None, reorg_energy: float = None,
                        temperature: float = None, spectral_density: str = None,
                        exponent: float = 1) -> np.ndarray:
    

    lindbladian = np.zeros((dims ** 2, dims ** 2), dtype=complex)

    if dynamics_model == 'local dephasing lindblad':
        assert deph_rate is not None, #Besoin d'une vitesse de déphasage
        #L'opérateur de Linblad  evalué pour chaque site, dans la base des sites.
        for site_j in range(dims):
            l_op = loc_deph_lindblad_op(dims, site_j)
            indiv_superop = lindblad_superop_sum_element(l_op)
            lindbladian += indiv_superop
        return deph_rate * lindbladian  # rad ps^-1

    # Vérification des entrées pour les modèles de thermalisation
    for var in [hamiltonian, cutoff_freq, reorg_energy, temperature,
                spectral_density]:
        assert var is not None, (
            'Need to pass a hamiltonian, cutoff frequency, scaling factor,'
            ' temperature, and spectral density for thermalising models.')
    if spectral_density == 'ohmic':
        assert exponent is not None, 'Need to pass the Ohmic exponent.'

    eigv, eigs = util.eigv(hamiltonian), util.eigs(hamiltonian)

    if dynamics_model == 'global thermalising lindblad':
        # L'opérateur de Linblad construit pour chaque paires différentes d'états et de valeurs p. 

        for state_a, state_b in permutations(range(dims), 2):
            omega_a, omega_b = eigv[state_a], eigv[state_b]
            k_a_to_b = bath.rate_constant_redfield((omega_a - omega_b),
                                                   deph_rate,
                                                   cutoff_freq,
                                                   reorg_energy,
                                                   temperature,
                                                   spectral_density,
                                                   exponent)
            if k_a_to_b == 0.:  
                continue
            l_op = glob_therm_lindblad_op(dims, state_a, state_b)
            indiv_superop = lindblad_superop_sum_element(l_op)
            indiv_superop = util.basis_change(indiv_superop, eigs, True)
            indiv_superop *= k_a_to_b
            lindbladian += indiv_superop
        return lindbladian  # rad ps^-1

    if dynamics_model == 'local thermalising lindblad':
        # L'opérateur de Linblad evalué pour chaque paire (x, y), où x
        # est la fréquance unique de gap entre les états propres de l'Hamiltonien
        # et y est le site dans le système quantique.
        gaps = eigv.reshape(dims, 1) - eigv
        unique = np.unique(gaps.flatten())  
        for unique, site_m in product(unique, range(dims)):
            k_omega = bath.rate_constant_redfield(unique,
                                                  deph_rate,
                                                  cutoff_freq,
                                                  reorg_energy,
                                                  temperature,
                                                  spectral_density,
                                                  exponent)
            l_op = loc_therm_lindblad_op(eigv, eigs, unique, site_m)
            indiv_superop = lindblad_superop_sum_element(l_op)
            lindbladian += k_omega * indiv_superop
        return lindbladian  # rad ps^-1

    raise NotImplementedError('Other lindblad dynamics models not yet'
                              ' implemented in quantum_HEOM. Choose from: '
                              + str(LINDBLAD_MODELS))